In [5]:
# FETCH HISTORICAL DATA

from torchtrade.data.cdd import CryptoDataDownload
import pandas as pd

cdd = CryptoDataDownload()

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
], axis=1)

# bitstamp_data.drop("BTC:volume", axis=1, inplace=True)
bitstamp_data

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,5.599300e+02
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,2.735800e+02
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,9.177900e+02
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,1.826200e+02
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1.260690e+03
...,...,...,...,...,...,...,...
57542,2024-12-06 20:00:00,1733515200,101642.00,102098.00,101251.00,101558.00,2.141584e+07
57543,2024-12-06 21:00:00,1733518800,101558.00,101643.00,100508.00,100578.00,4.900466e+06
57544,2024-12-06 22:00:00,1733522400,100579.00,100876.00,100518.00,100662.00,2.156769e+06
57545,2024-12-06 23:00:00,1733526000,100662.00,100744.00,99304.00,99898.00,1.023097e+07


In [3]:
# DEFINE EXCHANGES

from torchtrade.oms.exchanges import Exchange
from torchtrade.oms.services.execution.simulated import execute_order
from torchtrade.feed.core.base import Stream


bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC")
)

In [6]:
# DEFINE ADDITIONAL DATA (TA, etc)
import ta
from torchtrade.feed.core import Stream, DataFeed, NameSpace

bitstamp_btc = bitstamp_data.loc[:, bitstamp_data.columns.str.startswith("BTC")].copy()

ta.add_all_ta_features(
    bitstamp_btc,
    colprefix='BTC:',
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_steams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    
feed = DataFeed(bitstamp_steams)

feed.next()

{'bitstamp:/BTC:date': Timestamp('2018-05-15 06:00:00'),
 'bitstamp:/BTC:unix': 1526364000,
 'bitstamp:/BTC:open': 8733.86,
 'bitstamp:/BTC:high': 8796.68,
 'bitstamp:/BTC:low': 8707.28,
 'bitstamp:/BTC:close': 8740.99,
 'bitstamp:/BTC:volume': 559.93,
 'bitstamp:/BTC:volume_adi': -137.6651163311054,
 'bitstamp:/BTC:volume_obv': 559.93,
 'bitstamp:/BTC:volume_cmf': nan,
 'bitstamp:/BTC:volume_fi': nan,
 'bitstamp:/BTC:volume_em': nan,
 'bitstamp:/BTC:volume_sma_em': nan,
 'bitstamp:/BTC:volume_vpt': nan,
 'bitstamp:/BTC:volume_vwap': nan,
 'bitstamp:/BTC:volume_mfi': nan,
 'bitstamp:/BTC:volume_nvi': 1000.0,
 'bitstamp:/BTC:volatility_bbm': nan,
 'bitstamp:/BTC:volatility_bbh': nan,
 'bitstamp:/BTC:volatility_bbl': nan,
 'bitstamp:/BTC:volatility_bbw': nan,
 'bitstamp:/BTC:volatility_bbp': nan,
 'bitstamp:/BTC:volatility_bbhi': 0.0,
 'bitstamp:/BTC:volatility_bbli': 0.0,
 'bitstamp:/BTC:volatility_kcc': nan,
 'bitstamp:/BTC:volatility_kch': 8837.716666666667,
 'bitstamp:/BTC:volatility